### 1. RandomForestRegressor model

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import joblib

TRAIN_MODEL = True  # Set to False to skip training and use saved model
MODEL_PATH = 'rf_multi_model.joblib'
AVG_MODEL_PATH = 'rf_avg_rating_model.joblib'

# ========== Step 1: Load Data ==========
train_file = 'Blank rating calculation - Variant Model Train data.csv'
variant_file = 'Blank rating calculation - Variant Model Testing data.csv'
train_df = pd.read_csv(train_file)
variant_df = pd.read_csv(variant_file)

STAR_LIST = [1, 2, 3, 4, 5]

# ========== Step 2: Prepare Multi-Output Training Set ==========
train_pivot = train_df.pivot_table(
    index=['Product'],
    columns='Rating',
    values=['Review', 'No. of verifiied purchase', 'Percentage'],
    fill_value=0
)
train_pivot.columns = ['{}_{}'.format(col[0], int(col[1])) for col in train_pivot.columns]
train_pivot = train_pivot.reset_index()

X = train_pivot[[f'Review_{s}' for s in STAR_LIST] + [f'No. of verifiied purchase_{s}' for s in STAR_LIST]].values
y = train_pivot[[f'Percentage_{s}' for s in STAR_LIST]].values

# ========== Step 3: Train or Load Model ==========
if TRAIN_MODEL:
    rf_multi = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_multi.fit(X, y)
    joblib.dump(rf_multi, MODEL_PATH)
    print(f"Trained and saved multi-output model to {MODEL_PATH}")
else:
    rf_multi = joblib.load(MODEL_PATH)
    print(f"Loaded multi-output model from {MODEL_PATH}")

# ========== Helper: Enforce Weighted Star-Combined Percentage ==========
def enforce_weighted_star_relation(pred_table, reviews_table, combined_pct, n_iter=6):
    pred = pred_table.copy()
    for _ in range(n_iter):
        pred = pred / pred.sum(axis=1, keepdims=True) * 100
        for j in range(5):
            weight = reviews_table[:, j]
            total_weight = weight.sum()
            if total_weight == 0:
                continue
            weighted_avg = (pred[:, j] * weight).sum() / total_weight
            target = combined_pct[j]
            if weighted_avg == 0:
                scale = 1.0
            else:
                scale = target / weighted_avg
            pred[:, j] = pred[:, j] * scale
    pred = pred / pred.sum(axis=1, keepdims=True) * 100
    return pred

# ========== Step 4: Prepare Variant Data, Predict & Enforce Constraint ==========
variant_df['is_combined'] = variant_df['Variant'].astype(str).str.strip().str.lower() == 'combined'
results = []

for (prod, fam), group in variant_df.groupby(['Product', 'Family']):
    combined_row = group[group['is_combined']]
    variant_rows = group[~group['is_combined']]
    if combined_row.empty or variant_rows.empty:
        continue

    combined_total = combined_row['Total'].iloc[0]
    combined_pct = []
    for star in STAR_LIST:
        row = combined_row[combined_row['Rating'] == star]
        combined_pct.append(float(row['Percentage'].iloc[0]) if not row.empty else 0)
    combined_pct = np.array(combined_pct)

    variants = variant_rows['Variant'].unique()
    var_raw_pred = []
    var_reviews = []
    variant_meta = []

    for variant in variants:
        var_rows = variant_rows[variant_rows['Variant'] == variant]
        feat_review = []
        feat_verified = []
        star_reviews = []
        for star in STAR_LIST:
            row = var_rows[var_rows['Rating'] == star]
            feat_review.append(row['Review'].iloc[0] if not row.empty else 0)
            feat_verified.append(row['No. of verifiied purchase'].iloc[0] if not row.empty else 0)
            star_reviews.append(row['Review'].iloc[0] if not row.empty else 0)
        features = feat_review + feat_verified
        pred_raw = rf_multi.predict([features])[0]
        pred_raw = np.maximum(pred_raw, 0)
        var_raw_pred.append(pred_raw)
        var_reviews.append(star_reviews)
        variant_meta.append((variant, var_rows))

    var_raw_pred = np.array(var_raw_pred)
    var_reviews = np.array(var_reviews)
    var_final_pred = enforce_weighted_star_relation(var_raw_pred, var_reviews, combined_pct, n_iter=6)

    # Assign predictions back to rows
    for i, (variant, var_rows) in enumerate(variant_meta):
        for j, star in enumerate(STAR_LIST):
            row = var_rows[var_rows['Rating'] == star]
            if row.empty:
                base = var_rows.iloc[0].copy()
                base['Rating'] = star
                base['Review'] = 0
                base['No. of verifiied purchase'] = 0
            else:
                base = row.iloc[0].copy()
            base['pred_Percentage_raw'] = var_raw_pred[i, j]
            base['pred_Percentage_first_norm'] = var_final_pred[i, j]
            base['combined_Total'] = combined_total
            results.append(base)

pred_df = pd.DataFrame(results)

# ========== Step 5: Spreadsheet formula, per star per product, across variants, EXCLUDING combined ==========
def spreadsheet_starwise_norm(df):
    norm_col = pd.Series(0.0, index=df.index, dtype=float)
    mask_noncombined = ~df['Variant'].astype(str).str.strip().str.lower().eq('combined')
    df_noncombined = df[mask_noncombined]
    for (prod, rating), subdf in df_noncombined.groupby(['Product', 'Rating']):
        sum_reviews = subdf['Review'].sum()
        for idx, row in subdf.iterrows():
            val = (row['Review'] * row['pred_Percentage_first_norm']) / sum_reviews if sum_reviews > 0 else 0
            norm_col.at[idx] = val
    return norm_col

pred_df['pred_Percentage_final_norm'] = spreadsheet_starwise_norm(pred_df)

# ========== Step 6: Calculate pred_Total per star ==========
pred_df['pred_Total per star'] = (pred_df['pred_Percentage_final_norm'] / 100) * pred_df['combined_Total']
pred_df['pred_Total per star'] = pred_df['pred_Total per star'].round().astype(int)

# ========== Step 6B: Adjust to match combined variant's Total per star ==========
def adjust_pred_total_per_star(df, variant_df):
    df = df.copy()
    combined_totals = variant_df[variant_df['Variant'].astype(str).str.strip().str.lower() == 'combined'].set_index(['Product', 'Rating'])['Total per star'].to_dict()
    mask_noncombined = ~df['Variant'].astype(str).str.strip().str.lower().eq('combined')
    for (product, rating), group in df[mask_noncombined].groupby(['Product', 'Rating']):
        combined_total = combined_totals.get((product, rating), 0)
        variant_total = group['pred_Total per star'].sum()
        difference = combined_total - variant_total
        if abs(difference) < 10 and difference != 0:
            idx = group['pred_Total per star'].idxmax()
            df.loc[idx, 'pred_Total per star'] += difference
    return df

pred_df = adjust_pred_total_per_star(pred_df, variant_df)

# ========== Step 7: Calculate pred_Blank and pred_Total (per variant) ==========
pred_df['pred_Blank'] = pred_df['pred_Total per star'] - pred_df['Review']
pred_df['pred_Blank'] = pred_df['pred_Blank'].clip(lower=0)
pred_df['Total'] = pred_df.groupby(['Product', 'Variant'])['pred_Total per star'].transform('sum')

# ========== Step 8: Predict Average Rating (Second Model) ==========
avg_pivot = train_df.pivot_table(
    index=['Product'],
    columns='Rating',
    values=['Review', 'No. of verifiied purchase', 'Percentage', 'Total per star', 'Blank'],
    fill_value=0
)
avg_pivot.columns = ['{}_{}'.format(col[0], int(col[1])) for col in avg_pivot.columns]
avg_pivot = avg_pivot.reset_index()
avg_pivot['Total'] = avg_pivot[[f'Total per star_{s}' for s in STAR_LIST]].sum(axis=1)
avg_pivot['Average Rating'] = train_df.groupby('Product')['Average Rating'].first().values

avg_features = []
for prefix in ['Review', 'No. of verifiied purchase', 'Percentage', 'Total per star', 'Blank']:
    avg_features += [f'{prefix}_{s}' for s in STAR_LIST]
avg_features += ['Total']

if TRAIN_MODEL:
    avg_rating_model = RandomForestRegressor(n_estimators=100, random_state=42)
    avg_rating_model.fit(avg_pivot[avg_features], avg_pivot['Average Rating'])
    joblib.dump(avg_rating_model, AVG_MODEL_PATH)
    print(f"Trained and saved avg_rating_model to {AVG_MODEL_PATH}")
else:
    avg_rating_model = joblib.load(AVG_MODEL_PATH)
    print(f"Loaded avg_rating_model from {AVG_MODEL_PATH}")

pred_df['Total per star'] = pred_df['pred_Total per star']
for prefix in ['Review', 'No. of verifiied purchase', 'pred_Percentage_first_norm', 'pred_Total per star', 'pred_Blank']:
    for s in STAR_LIST:
        pred_df[f'{prefix}_{s}'] = pred_df.groupby(['Product', 'Variant'])[prefix].transform(lambda x: list(x) if len(x)==5 else [0]*5)

avg_pred_features = []
for prefix in ['Review', 'No. of verifiied purchase', 'pred_Percentage_first_norm', 'pred_Total per star', 'pred_Blank']:
    avg_pred_features += [f'{prefix}_{s}' for s in STAR_LIST]
avg_pred_features += ['Total']

for (prod, variant), group in pred_df.groupby(['Product', 'Variant']):
    if len(group) != 5:
        continue
    row = {}
    for prefix in ['Review', 'No. of verifiied purchase', 'pred_Percentage_first_norm', 'pred_Total per star', 'pred_Blank']:
        vals = group[prefix].values
        for i, s in enumerate(STAR_LIST):
            row[f'{prefix}_{s}'] = vals[i]
    row['Total'] = group['Total'].iloc[0]
    pred = avg_rating_model.predict([list(row.values())])[0]
    pred_df.loc[group.index, 'pred_Average Rating'] = pred

# ========== Step 9: Merge with Combined Rows, Preserve Order ==========
output_columns = [
    'Family', 'Product', 'Variant', 'Rating', 'Review', 'No. of verifiied purchase',
    'pred_Percentage_raw', 'pred_Percentage_first_norm', 'pred_Percentage_final_norm',
    'pred_Total per star', 'pred_Blank', 'Total', 'pred_Average Rating'
]

combined_output = variant_df[variant_df['is_combined']].copy()
combined_output['pred_Percentage_raw'] = combined_output['Percentage']
combined_output['pred_Percentage_first_norm'] = combined_output['Percentage']
combined_output['pred_Percentage_final_norm'] = combined_output['Percentage']
combined_output['pred_Total per star'] = combined_output['Total per star']
combined_output['pred_Blank'] = combined_output['Blank']
combined_output['pred_Average Rating'] = combined_output['Average Rating']
combined_output['Total'] = combined_output['Total']
combined_output = combined_output[output_columns]

final_output = pd.concat([pred_df[output_columns], combined_output], ignore_index=True)

# Round pred_Average Rating to 1 decimal
final_output['pred_Average Rating'] = final_output['pred_Average Rating'].round(1)

# ========== Step 10: Save Output ==========
final_output.to_csv('variant_predictions_spreadsheet_style_norm.csv', index=False)
print("Prediction complete. Output saved to 'variant_predictions_spreadsheet_style_norm.csv'")


Trained and saved multi-output model to rf_multi_model.joblib
Trained and saved avg_rating_model to rf_avg_rating_model.joblib
Prediction complete. Output saved to 'variant_predictions_spreadsheet_style_norm.csv'


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: Use

In [2]:
final_output.head()

,Family,Product,Variant,Rating,Review,No. of verifiied purchase,pred_Percentage_raw,pred_Percentage_first_norm,pred_Percentage_final_norm,pred_Total per star,pred_Blank,Total,pred_Average Rating
0,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,1,471,443,6.11,7.141713,6.795449,1059.0,588.0,14448.0,4.3
1,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,2,190,174,2.71,3.062538,2.852364,444.0,254.0,14448.0,4.3
2,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,3,238,228,5.73,5.115313,4.718777,734.0,496.0,14448.0,4.3
3,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,4,417,395,14.62,12.186614,11.497326,1789.0,1372.0,14448.0,4.3
4,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,5,1822,1739,70.83,72.493821,66.877844,10422.0,8600.0,14448.0,4.3


### 2. MultiOutputRegressor with GradientBoostingRegressor

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.multioutput import MultiOutputRegressor
import joblib

MODEL_PATH = 'gboost_multi_model.joblib'
AVG_MODEL_PATH = 'gboost_avg_rating_model.joblib'

train_file = 'Blank rating calculation - Variant Model Train data.csv'
variant_file = 'Blank rating calculation - Variant Model Testing data.csv'
train_df = pd.read_csv(train_file)
variant_df = pd.read_csv(variant_file)

STAR_LIST = [1, 2, 3, 4, 5]

# === Step 2: Prepare Multi-Output Training Set ===
train_pivot = train_df.pivot_table(
    index=['Product'],
    columns='Rating',
    values=['Review', 'No. of verifiied purchase', 'Percentage'],
    fill_value=0
)
train_pivot.columns = ['{}_{}'.format(col[0], int(col[1])) for col in train_pivot.columns]
train_pivot = train_pivot.reset_index()

X = train_pivot[[f'Review_{s}' for s in STAR_LIST] + [f'No. of verifiied purchase_{s}' for s in STAR_LIST]].values
y = train_pivot[[f'Percentage_{s}' for s in STAR_LIST]].values

# === Step 3: Train or Load Model ===
gboost_multi = MultiOutputRegressor(GradientBoostingRegressor(n_estimators=100, random_state=42))
gboost_multi.fit(X, y)
joblib.dump(gboost_multi, MODEL_PATH)
print(f"Trained and saved multi-output GradientBoostingRegressor model to {MODEL_PATH}")

# === Helper: Enforce Weighted Star-Combined Percentage ===
def enforce_weighted_star_relation(pred_table, reviews_table, combined_pct, n_iter=6):
    pred = pred_table.copy()
    for _ in range(n_iter):
        pred = pred / pred.sum(axis=1, keepdims=True) * 100
        for j in range(5):
            weight = reviews_table[:, j]
            total_weight = weight.sum()
            if total_weight == 0:
                continue
            weighted_avg = (pred[:, j] * weight).sum() / total_weight
            target = combined_pct[j]
            scale = target / weighted_avg if weighted_avg != 0 else 1.0
            pred[:, j] = pred[:, j] * scale
    pred = pred / pred.sum(axis=1, keepdims=True) * 100
    return pred

# === Step 4+: Predict & Enforce Constraint ===
variant_df['is_combined'] = variant_df['Variant'].astype(str).str.strip().str.lower() == 'combined'
results = []
for (prod, fam), group in variant_df.groupby(['Product', 'Family']):
    combined_row = group[group['is_combined']]
    variant_rows = group[~group['is_combined']]
    if combined_row.empty or variant_rows.empty:
        continue

    combined_total = combined_row['Total'].iloc[0]
    combined_pct = []
    for star in STAR_LIST:
        row = combined_row[combined_row['Rating'] == star]
        combined_pct.append(float(row['Percentage'].iloc[0]) if not row.empty else 0)
    combined_pct = np.array(combined_pct)

    variants = variant_rows['Variant'].unique()
    var_raw_pred = []
    var_reviews = []
    variant_meta = []

    for variant in variants:
        var_rows = variant_rows[variant_rows['Variant'] == variant]
        feat_review = []
        feat_verified = []
        star_reviews = []
        for star in STAR_LIST:
            row = var_rows[var_rows['Rating'] == star]
            feat_review.append(row['Review'].iloc[0] if not row.empty else 0)
            feat_verified.append(row['No. of verifiied purchase'].iloc[0] if not row.empty else 0)
            star_reviews.append(row['Review'].iloc[0] if not row.empty else 0)
        features = feat_review + feat_verified
        pred_raw = gboost_multi.predict([features])[0]
        pred_raw = np.maximum(pred_raw, 0)
        var_raw_pred.append(pred_raw)
        var_reviews.append(star_reviews)
        variant_meta.append((variant, var_rows))

    var_raw_pred = np.array(var_raw_pred)
    var_reviews = np.array(var_reviews)
    var_final_pred = enforce_weighted_star_relation(var_raw_pred, var_reviews, combined_pct, n_iter=6)

    for i, (variant, var_rows) in enumerate(variant_meta):
        for j, star in enumerate(STAR_LIST):
            row = var_rows[var_rows['Rating'] == star]
            if row.empty:
                base = var_rows.iloc[0].copy()
                base['Rating'] = star
                base['Review'] = 0
                base['No. of verifiied purchase'] = 0
            else:
                base = row.iloc[0].copy()
            base['pred_Percentage_raw'] = var_raw_pred[i, j]
            base['pred_Percentage_first_norm'] = var_final_pred[i, j]
            base['combined_Total'] = combined_total
            results.append(base)
pred_df = pd.DataFrame(results)

def spreadsheet_starwise_norm(df):
    norm_col = pd.Series(0.0, index=df.index, dtype=float)
    mask_noncombined = ~df['Variant'].astype(str).str.strip().str.lower().eq('combined')
    df_noncombined = df[mask_noncombined]
    for (prod, rating), subdf in df_noncombined.groupby(['Product', 'Rating']):
        sum_reviews = subdf['Review'].sum()
        for idx, row in subdf.iterrows():
            val = (row['Review'] * row['pred_Percentage_first_norm']) / sum_reviews if sum_reviews > 0 else 0
            norm_col.at[idx] = val
    return norm_col

pred_df['pred_Percentage_final_norm'] = spreadsheet_starwise_norm(pred_df)
pred_df['pred_Total per star'] = (pred_df['pred_Percentage_final_norm'] / 100) * pred_df['combined_Total']
pred_df['pred_Total per star'] = pred_df['pred_Total per star'].round().astype(int)

def adjust_pred_total_per_star(df, variant_df):
    df = df.copy()
    combined_totals = variant_df[variant_df['Variant'].astype(str).str.strip().str.lower() == 'combined'].set_index(['Product', 'Rating'])['Total per star'].to_dict()
    mask_noncombined = ~df['Variant'].astype(str).str.strip().str.lower().eq('combined')
    for (product, rating), group in df[mask_noncombined].groupby(['Product', 'Rating']):
        combined_total = combined_totals.get((product, rating), 0)
        variant_total = group['pred_Total per star'].sum()
        difference = combined_total - variant_total
        if abs(difference) < 10 and difference != 0:
            idx = group['pred_Total per star'].idxmax()
            df.loc[idx, 'pred_Total per star'] += difference
    return df

pred_df = adjust_pred_total_per_star(pred_df, variant_df)
pred_df['pred_Blank'] = pred_df['pred_Total per star'] - pred_df['Review']
pred_df['pred_Blank'] = pred_df['pred_Blank'].clip(lower=0)
pred_df['Total'] = pred_df.groupby(['Product', 'Variant'])['pred_Total per star'].transform('sum')

# ========== Step 8: Predict Average Rating (Second Model) ==========
avg_pivot = train_df.pivot_table(
    index=['Product'],
    columns='Rating',
    values=['Review', 'No. of verifiied purchase', 'Percentage', 'Total per star', 'Blank'],
    fill_value=0
)
avg_pivot.columns = ['{}_{}'.format(col[0], int(col[1])) for col in avg_pivot.columns]
avg_pivot = avg_pivot.reset_index()
avg_pivot['Total'] = avg_pivot[[f'Total per star_{s}' for s in STAR_LIST]].sum(axis=1)
avg_pivot['Average Rating'] = train_df.groupby('Product')['Average Rating'].first().values

avg_features = []
for prefix in ['Review', 'No. of verifiied purchase', 'Percentage', 'Total per star', 'Blank']:
    avg_features += [f'{prefix}_{s}' for s in STAR_LIST]
avg_features += ['Total']

avg_rating_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
avg_rating_model.fit(avg_pivot[avg_features], avg_pivot['Average Rating'])
joblib.dump(avg_rating_model, AVG_MODEL_PATH)
print(f"Trained and saved avg_rating_model to {AVG_MODEL_PATH}")

pred_df['Total per star'] = pred_df['pred_Total per star']
for prefix in ['Review', 'No. of verifiied purchase', 'pred_Percentage_first_norm', 'pred_Total per star', 'pred_Blank']:
    for s in STAR_LIST:
        pred_df[f'{prefix}_{s}'] = pred_df.groupby(['Product', 'Variant'])[prefix].transform(lambda x: list(x) if len(x)==5 else [0]*5)

avg_pred_features = []
for prefix in ['Review', 'No. of verifiied purchase', 'pred_Percentage_first_norm', 'pred_Total per star', 'pred_Blank']:
    avg_pred_features += [f'{prefix}_{s}' for s in STAR_LIST]
avg_pred_features += ['Total']

for (prod, variant), group in pred_df.groupby(['Product', 'Variant']):
    if len(group) != 5:
        continue
    row = {}
    for prefix in ['Review', 'No. of verifiied purchase', 'pred_Percentage_first_norm', 'pred_Total per star', 'pred_Blank']:
        vals = group[prefix].values
        for i, s in enumerate(STAR_LIST):
            row[f'{prefix}_{s}'] = vals[i]
    row['Total'] = group['Total'].iloc[0]
    pred = avg_rating_model.predict([list(row.values())])[0]
    pred_df.loc[group.index, 'pred_Average Rating'] = pred

output_columns = [
    'Family', 'Product', 'Variant', 'Rating', 'Review', 'No. of verifiied purchase',
    'pred_Percentage_raw', 'pred_Percentage_first_norm', 'pred_Percentage_final_norm',
    'pred_Total per star', 'pred_Blank', 'Total', 'pred_Average Rating'
]

combined_output = variant_df[variant_df['is_combined']].copy()
combined_output['pred_Percentage_raw'] = combined_output['Percentage']
combined_output['pred_Percentage_first_norm'] = combined_output['Percentage']
combined_output['pred_Percentage_final_norm'] = combined_output['Percentage']
combined_output['pred_Total per star'] = combined_output['Total per star']
combined_output['pred_Blank'] = combined_output['Blank']
combined_output['pred_Average Rating'] = combined_output['Average Rating']
combined_output['Total'] = combined_output['Total']
combined_output = combined_output[output_columns]

final_output = pd.concat([pred_df[output_columns], combined_output], ignore_index=True)
final_output['pred_Average Rating'] = final_output['pred_Average Rating'].round(1)
final_output.to_csv('variant_predictions_gboost.csv', index=False)
print("Prediction complete. Output saved to 'variant_predictions_gboost.csv'")


Trained and saved multi-output GradientBoostingRegressor model to gboost_multi_model.joblib
Trained and saved avg_rating_model to gboost_avg_rating_model.joblib
Prediction complete. Output saved to 'variant_predictions_gboost.csv'


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but GradientBoostingRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/

In [4]:
final_output.head()

,Family,Product,Variant,Rating,Review,No. of verifiied purchase,pred_Percentage_raw,pred_Percentage_first_norm,pred_Percentage_final_norm,pred_Total per star,pred_Blank,Total,pred_Average Rating
0,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,1,471,443,6.681123,7.234639,6.883869,1073.0,602.0,14447.0,4.3
1,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,2,190,174,3.896681,3.121691,2.907457,453.0,263.0,14447.0,4.3
2,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,3,238,228,5.153731,5.167415,4.766840,741.0,503.0,14447.0,4.3
3,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,4,417,395,16.038235,12.244673,11.552101,1796.0,1379.0,14447.0,4.3
4,Ryobi - Competitor,Blower,DEWALT 20V MAX Compact Jobsite Blower,5,1822,1739,62.330626,72.231583,66.635921,10384.0,8562.0,14447.0,4.3


### Testing the models

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def evaluate_model_prediction(
    pred_csv_path,
    gt_csv_path,
    model_name='model'
):
    pred = pd.read_csv(pred_csv_path)
    gt = pd.read_csv(gt_csv_path)
    key_cols = ['Product', 'Variant', 'Rating']
    merged = pd.merge(pred, gt, on=key_cols, suffixes=('_pred', '_gt'))

    metrics = {}

    # Per-star Percentage
    if 'pred_Percentage_final_norm' in merged.columns and 'Percentage' in merged.columns:
        y_true = merged['Percentage']
        y_pred = merged['pred_Percentage_final_norm']
        metrics['percentage_mae'] = mean_absolute_error(y_true, y_pred)
        metrics['percentage_rmse'] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics['percentage_r2'] = r2_score(y_true, y_pred)
    else:
        print(f"[{model_name}] Skipping percentage error: column missing.")

    # Per-star Total per star
    if 'pred_Total per star' in merged.columns and 'Total per star' in merged.columns:
        y_true = merged['Total per star']
        y_pred = merged['pred_Total per star']
        metrics['total_per_star_mae'] = mean_absolute_error(y_true, y_pred)
        metrics['total_per_star_rmse'] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics['total_per_star_r2'] = r2_score(y_true, y_pred)
    else:
        print(f"[{model_name}] Skipping total per star error: column missing.")

    # Average Rating (one row per Product/Variant)
    if 'pred_Average Rating' in merged.columns and 'Average Rating' in merged.columns:
        avg_pred = merged.drop_duplicates(subset=['Product', 'Variant'])[['pred_Average Rating', 'Average Rating']]
        y_true = avg_pred['Average Rating']
        y_pred = avg_pred['pred_Average Rating']
        metrics['avg_rating_mae'] = mean_absolute_error(y_true, y_pred)
        metrics['avg_rating_rmse'] = np.sqrt(mean_squared_error(y_true, y_pred))
        metrics['avg_rating_r2'] = r2_score(y_true, y_pred)
    else:
        print(f"[{model_name}] Skipping average rating error: column missing.")

    return metrics

def batch_evaluate(
    pred_gt_pairs, # List of (model_name, pred_csv_path, gt_csv_path)
    output_csv='model_evaluation_results.csv'
):
    rows = []
    for model_name, pred_csv, gt_csv in pred_gt_pairs:
        print(f"\n=== Evaluating {model_name} ===")
        metrics = evaluate_model_prediction(pred_csv, gt_csv, model_name=model_name)
        metrics['model_name'] = model_name

        # Composite metrics
        rmse_keys = [k for k in metrics.keys() if k.endswith('_rmse')]
        mae_keys  = [k for k in metrics.keys() if k.endswith('_mae')]
        r2_keys   = [k for k in metrics.keys() if k.endswith('_r2')]
        # Calculate only if at least one metric is present (avoid division by zero)
        metrics['composite_rmse'] = np.mean([metrics[k] for k in rmse_keys if not pd.isnull(metrics[k])]) if rmse_keys else np.nan
        metrics['composite_mae']  = np.mean([metrics[k] for k in mae_keys if not pd.isnull(metrics[k])]) if mae_keys else np.nan
        metrics['composite_r2']   = np.mean([metrics[k] for k in r2_keys if not pd.isnull(metrics[k])]) if r2_keys else np.nan

        rows.append(metrics)

    df = pd.DataFrame(rows)
    df.to_csv(output_csv, index=False)
    print(f"\nSaved metrics to {output_csv}")
    return df


In [6]:
# Error and accuracy calculation function
pred_gt_pairs = [
    ("RandomForest", "variant_predictions_spreadsheet_style_norm.csv", "Blank rating calculation - Variant Ground Truth.csv"),
    ("MultiOutputRegressor with GradientBoostingRegressor", "variant_predictions_gboost.csv", "Blank rating calculation - Variant Ground Truth.csv")
    # Add more models as needed...
]
results_df = batch_evaluate(pred_gt_pairs)

print("MAE (Mean Square Error) - penalizes larger errors more heavily (good if outliers matter to you)")
print("RMSE (Root Mean Squared Error) - is more robust to outliers and is the “average error” in the same units as your data")
print("R2 (coefficient of determination) - Ranges from 0 to 1, with 1.0 being perfect prediction. Can be interpreted as the proportion of variance explained")
results_df.head()


=== Evaluating RandomForest ===

=== Evaluating MultiOutputRegressor with GradientBoostingRegressor ===

Saved metrics to model_evaluation_results.csv
MAE (Mean Square Error) - penalizes larger errors more heavily (good if outliers matter to you)
RMSE (Root Mean Squared Error) - is more robust to outliers and is the “average error” in the same units as your data
R2 (coefficient of determination) - Ranges from 0 to 1, with 1.0 being perfect prediction. Can be interpreted as the proportion of variance explained


,percentage_mae,percentage_rmse,percentage_r2,total_per_star_mae,total_per_star_rmse,total_per_star_r2,avg_rating_mae,avg_rating_rmse,avg_rating_r2,model_name,composite_rmse,composite_mae,composite_r2
0,8.005766,17.649628,0.430555,279.44,928.528169,0.964295,0.07,0.114018,0.856195,RandomForest,315.430605,95.838589,0.750348
1,8.005559,17.627335,0.431992,281.76,957.028735,0.962069,0.08,0.126491,0.823009,MultiOutputRegressor with GradientBoostingRegr...,324.927520,96.615186,0.739023
